Validate the Assertions

In [25]:
#read the dataset
import pandas as pd
df = pd.read_csv("C:/Users/sriva/Downloads/Hwy26Crashes2019_S23.csv")
print(df.head()) 

   Crash ID  Record Type  Vehicle ID  Participant ID  \
0   1809119            1         NaN             NaN   
1   1809119            2   3409578.0             NaN   
2   1809119            3   3409578.0       3887884.0   
3   1809119            2   3409579.0             NaN   
4   1809119            3   3409579.0       3887885.0   

   Participant Display Seq#  Vehicle Coded Seq#  Participant Vehicle Seq#  \
0                       NaN                 NaN                       NaN   
1                       NaN                 1.0                       NaN   
2                       1.0                 1.0                       1.0   
3                       NaN                 2.0                       NaN   
4                       2.0                 2.0                       1.0   

   Serial #  Crash Month  Crash Day  ...  Participant Cause 2 Code  \
0   99992.0          3.0       14.0  ...                       NaN   
1       NaN          NaN        NaN  ...                    

In [26]:
#existence assertions
grouped_df = df.groupby('Crash ID')[['Crash Month', 'Crash Day', 'Crash Year']].agg('first')
#print(grouped_df)

#Check if Crash Month,Crash Day, and Crash Year are null for each Crash ID
is_null = grouped_df.isnull().all(axis=1)

#Get only data that are not null 
df_null_dates = grouped_df[is_null]

#Print the data
print(df_null_dates)

Empty DataFrame
Columns: [Crash Month, Crash Day, Crash Year]
Index: []


In [27]:
#limit assertions
#Group by Crash ID
grouped_df = df.groupby('Crash ID')[['Crash Month', 'Crash Day', 'Crash Year']].agg('first')

#Check if Crash Month,Crash Day, and Crash Year columns are null for each Crash ID
is_null = grouped_df.notnull().any(axis=1)

#Get data where the date columns are null 
df_null_dates = grouped_df[is_null]

#Find the data thare not in the specified range
invalid_day_month = ~(df_null_dates['Crash Day'].between(1, 31, inclusive='both')) | ~(df_null_dates['Crash Month'].between(1, 12, inclusive='both'))
invalid_dates = df_null_dates[invalid_day_month]

#Print the data
print(invalid_dates)

Empty DataFrame
Columns: [Crash Month, Crash Day, Crash Year]
Index: []


In [28]:
#intra-record assertions
#Group by Participant ID and find any missing age
missing_age = df.groupby('Participant ID')['Age'].apply(lambda x: x.isnull().any())
#print(missing_age)

if not missing_age.any():
    print("Each participant has an age.")
else:
    print("Not each participant has an age.")

    missing_participants = missing_age[missing_age]

    #Print the Participants with missing age
    print("Participant IDs with missing ages:")
    print(missing_participants.index.tolist())

Not each participant has an age.
Participant IDs with missing ages:
[3888074.0, 3890873.0, 3917895.0, 3917896.0, 3917897.0, 3920554.0, 3934436.0, 3950096.0, 3950097.0, 3952798.0, 3952799.0, 3956990.0, 3964941.0, 3973419.0, 3974799.0, 3975837.0, 3977088.0, 3977520.0, 3979355.0, 3979695.0]


In [29]:
#inter-record check assertions
#Group Participant ID based on Crash ID and find missing values
missing_crash_id = df.groupby('Participant ID')['Crash ID'].apply(lambda x: x.isnull().any())

#Group Participant ID based on Vehicle ID and find missing values
missing_vehicle_id = df.groupby('Participant ID')['Vehicle ID'].apply(lambda x: x.isnull().any())

if not missing_crash_id.any():
    print("Every participant was involved in a known crash.")
else:
    print("Not every participant was involved in a known crash.")

if not missing_vehicle_id.any():
    print("Every participant is associated with a vehicle ID.")
else:
    print("Not every participant is associated with a vehicle ID.")
    

Every participant was involved in a known crash.
Every participant is associated with a vehicle ID.


In [30]:
#summary assertions
#Get the data that does not have crash year as 2019
invalid_year = df_null_dates['Crash Year'] != 2019

invalid_dates_year = df_null_dates[invalid_year]

print(invalid_dates_year)

grouped_df = df.groupby('Crash ID')['Highway Number'].agg('first')

#Get the data that does not have highway number as 26
invalid_highway = grouped_df[grouped_df != 26]

print(invalid_highway)

Empty DataFrame
Columns: [Crash Month, Crash Day, Crash Year]
Index: []
Series([], Name: Highway Number, dtype: float64)


In [31]:
#statistical assertion
#Get the distribution of data across all days of the week
day_of_week_counts = df['Week Day Code'].value_counts(normalize=True)
print("Distribution of crashes across all days of the week:")
print(day_of_week_counts.sort_index())
std_deviation = day_of_week_counts.std()
print("Standard Deviation of the distribution across days:", std_deviation)

#Get the distribution of data across all days of the week
hour_counts = df['Crash Hour'].value_counts(normalize=True)
print("\nDistribution of crashes across all hours:")
print(hour_counts.sort_index())
std_deviation = hour_counts.std()
print("Standard Deviation of the distribution across hours:", std_deviation)

Distribution of crashes across all days of the week:
Week Day Code
1.0    0.118110
2.0    0.139764
3.0    0.147638
4.0    0.145669
5.0    0.133858
6.0    0.151575
7.0    0.163386
Name: proportion, dtype: float64
Standard Deviation of the distribution across days: 0.014324485560474736

Distribution of crashes across all hours:
Crash Hour
0.0     0.021654
1.0     0.009843
2.0     0.009843
3.0     0.007874
4.0     0.003937
5.0     0.017717
6.0     0.015748
7.0     0.037402
8.0     0.051181
9.0     0.055118
10.0    0.045276
11.0    0.035433
12.0    0.068898
13.0    0.057087
14.0    0.066929
15.0    0.102362
16.0    0.074803
17.0    0.098425
18.0    0.055118
19.0    0.045276
20.0    0.049213
21.0    0.035433
22.0    0.013780
23.0    0.013780
99.0    0.007874
Name: proportion, dtype: float64
Standard Deviation of the distribution across hours: 0.028223157036256344


Resolve the Violations and Transform the Data

In [32]:
df['Age'] = df['Age'].fillna('00')

missing_age = df['Age'].isnull().any()
if not missing_age:
    print("Missing age values filled with '00' successfully.")
else:
    print("Failed to fill missing age values with '00'.")


Missing age values filled with '00' successfully.


In [50]:
crash_dataset = ['Crash ID', 'Crash Month', 'Crash Day','Crash Year','Week Day Code','Crash Hour','County Code','City Section ID', 'Urban Area Code', 'Functional Class Code', 'NHS Flag', 'Highway Number', 'Highway Suffix', 'Roadway Number', 'Highway Component', 'Mileage Type', 'Connection Number', 'Linear Reference System (LRS)', 'Latitude Degrees', 'Latitude Minutes', 'Latitude Seconds', 'Longitude Degrees', 'Longitude Minutes', 'Longitude Seconds', 'Latitude (Decimal Degrees)', 'Longitude (Decimal Degrees)', 'Special Jurisdiction', 'Jurisdiction Group', 'Street Number', 'Nearest Intersecting Street Number', 'Intersection Sequence Number', 'Distance from Intersection', 'Direction From Intersection', 'Milepoint', 'Posted Speed Limit', 'Road Character', 'Off Roadway Flag', 'Intersection Type', 'Intersection Related Flag', 'Roundabout Flag', 'Driveway Related Flag', 'Number of Lanes', 'Number of Turning Legs', 'Median Type', 'Impact Location', 'Crash Type', 'Collision Type', 'Crash Severity', 'Weather Condition', 'Road Surface Condition', 'Light Condition', 'Traffic Control Device (TCD)', 'TCD Functional Flag', 'Investigating Agency', 'Crash Level Event 1 Code', 'Crash Level Event 2 Code', 'Crash Level Event 3 Code', 'Crash Level Cause 1 Code', 'Crash Level Cause 2 Code', 'Crash Level Cause 3 Code', 'School Zone Indicator', 'Work Zone Indicator', 'Secondary Crash Indicator', 'Alcohol-Involved Flag', 'Drugs Involved Flag', 'Crash Marijuana Involved Flag', 'Speed Involved Flag', 'Crash Level Hit & Run Flag', 'Population Range Code', 'Road Control', 'Route Type', 'Route Number', 'Region ID', 'District ID', 'Segment Marker ID', 'Segment Point LRS Measure', 'Unlocatable Flag', 'Total Vehicle Count', 'Total Fatality Count', 'Total Suspected Serious Injury (A) Count', 'Total Suspected Minor Injury (B) Count', 'Total Possible Injury (C) Count', 'Total Non-Fatal Injury Count', 'Total Un-Injured  Children Age 00-04', 'Total Un-Injured Persons', 'Total Pedestrian Count', 'Total Pedestrian Fatality Count', 'Total Pedestrian Non-Fatal Injury Count', 'Total Pedalcyclist Count', 'Total Pedalcyclist Fatality Count', 'Total Pedalcyclist Non-Fatal Injury Count', 'Total Unknown Non-Motorist Count', 'Total Unknown Non-Motorist Fatality Count', 'Total Unknown Non-Motorist Injury Count', 'Total Vehicle Occupant Count', 'Total Count of Persons Involved', 'Total Persons Not Using Safety Equipment', 'Total Persons Safety Equipment Use Unknown']
vehicle_dataset = ['Crash ID','Vehicle ID', 'Vehicle Ownership Code', 'Vehicle Special Use Code', 'Vehicle Type Code', 'Emergency Use Flag', 'Number of Trailers', 'Vehicle Movement Code', 'Vehicle Travel Direction From', 'Vehicle Travel Direction To', 'Vehicle Action Code', 'Vehicle Cause 1 Code', 'Vehicle Cause 2 Code', 'Vehicle Cause 3 Code', 'Vehicle Event 1 Code', 'Vehicle Event 2 Code', 'Vehicle Event 3 Code', 'Vehicle Exceeded Posted Speed Flag', 'Vehicle Hit & Run Flag', 'Safety Equipment Used Quantity', 'Safety Equipment Un-used Quantity', 'Safety Equipment Use Unknown Quantity', 'Vehicle Occupant Count', 'Vehicle Striking Flag']
participant_dataset = ['Crash ID', 'Vehicle ID', 'Participant ID','Participant Type Code', 'Participant Hit & Run Flag', 'Public Employee Flag', 'Sex', 'Age', 'Driver License Status', 'Driver Residence Status', 'Injury Severity', 'Participant Safety Equipment Use Code', 'Airbag Deployment', 'Non-Motorist Movement Code', 'Non-Motorist Travel Direction From', 'Non-Motorist Travel Direction To', 'Non-Motorist Location', 'Participant Action', 'Participant Error 1 Code', 'Participant Error 2 Code', 'Participant Error 3 Code', 'Participant Cause 1 Code', 'Participant Cause 2 Code', 'Participant Cause 3 Code', 'Participant Event 1 Code', 'Participant Event 2 Code', 'Participant Event 3 Code', 'BAC Test Results Code', 'Alcohol Use Reported', 'Drug Use Reported', 'Participant Marijuana Use Reported', 'Participant Striker Flag']

#Split the dataset into three datasets for crashes,vehicles and participants
crash = df[crash_dataset].drop_duplicates(subset=['Crash ID']).copy()
vehicle = df[vehicle_dataset].dropna(subset=['Vehicle ID']).drop_duplicates(subset=['Crash ID', 'Vehicle ID']).copy()
participant = df[participant_dataset].dropna(subset=['Participant ID']).drop_duplicates(subset=['Crash ID', 'Vehicle ID','Participant ID']).copy()

vehicle = vehicle[vehicle['Vehicle ID'] != 0]
participant = participant[participant['Participant ID'] != 0]
# Print the first few rows of each dataset to verify
print("Crash Dataset:")
print(crash.head())
print("\nVehicle Dataset:")
print(vehicle.head())
print("\nPartcipant Dataset:")
print(participant.head())

Crash Dataset:
    Crash ID  Crash Month  Crash Day  Crash Year  Week Day Code  Crash Hour  \
0    1809119          3.0       14.0      2019.0            5.0        14.0   
5    1809229          3.0        3.0      2019.0            1.0         2.0   
9    1809637          5.0       17.0      2019.0            6.0        99.0   
12   1810874          4.0        7.0      2019.0            1.0         0.0   
16   1812266          7.0        8.0      2019.0            2.0        14.0   

    County Code  City Section ID  Urban Area Code  Functional Class Code  ...  \
0          26.0              0.0             57.0                   14.0  ...   
5           3.0            185.0             68.0                   14.0  ...   
9           3.0              0.0              0.0                    2.0  ...   
12         26.0            245.0             57.0                   14.0  ...   
16         26.0            245.0             57.0                   14.0  ...   

    Total Pedalcyclist 

In [52]:
crash.to_csv('C:/Users/sriva/OneDrive/Documents/Data Eng/crash_dataset.csv', index=False)
vehicle.to_csv('C:/Users/sriva/OneDrive/Documents/Data Eng/vehicle_dataset.csv', index=False)
participant.to_csv('C:/Users/sriva/OneDrive/Documents/Data Eng/participant_dataset.csv', index=False)